# Analysis of Duval County (FL) Court Data

In [1]:
import pandas as pd
import numpy as np

## Load case data from Duval County

In [2]:
cases = pd.read_excel(
    "../data/Cases_w_InvoluntaryPlacementPetitions_2016-11-07.xlsx",
    sheetname='Cases with Petitioners')

In [3]:
len(cases)

10738

In [4]:
cases.head(2)

,CaseNumber,PetitionerAddressPart1,PetitionerAddressPart2,PetitionerCity,PetitionerState,PetitionerZipCode,PetitionerRelationship
0,16-2008-MH-000431-XXXX-MA,800 PRUDENTIAL DR.,NaN,JACKSONVILLE,FL,32207,ADMINISTRATOR
1,16-2008-MH-000432-XXXX-MA,800 PRUDENTIAL DR.,NaN,JACKSONVILLE,FL,32207,ADMINISTRATOR BAPTIST MEDICAL CENTER


In [5]:
cases.CaseNumber.nunique()

10725

Checked that duplicate case numbers do not have different addresses for the petitioner.

- Need to exclude `16-2009-MH-000250-XXXX-MA` because it lists both `11820 BEACH BLVD` and 
    `MHCJ 3333 W 20 TH ST` as the petitioner addresses. This does not affect the calculations
    for River Point, located at 6300 BEACH BLVD.

In [6]:
cases = cases[~cases["CaseNumber"].str.contains('16-2009-MH-000250-XXXX-MA')]

In [7]:
cases["FileYear"] = cases["CaseNumber"].str.extract('-(....)-', expand=True)

In [8]:
cases["FileYear"] = cases["FileYear"].astype(int)

In [9]:
cases_2009_2015 = cases[
    (cases["FileYear"] >= 2009) &
    (cases["FileYear"] <= 2015)
].copy()

In [10]:
pd.DataFrame(cases_2009_2015.groupby("FileYear")["CaseNumber"].nunique())

,CaseNumber
FileYear,
2009,702
2010,980
2011,1148
2012,1731
2013,2050
2014,1732
2015,973


### Clean up addresses 

In [11]:
cases_2009_2015["address_clean"] = np.nan

In [12]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("11820 BEACH", case=False, na=False)),
"address_clean"] = '11820 BEACH BLVD JACKSONVILLE, FL 32246'

In [13]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("^333", case=False, na=False)) &
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("20", case=False, na=False)) &
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("W", case=False, na=False)),
"address_clean"] = '3333 WEST 20TH ST JACKSONVILLE, FL 32209'

In [14]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("3333 W", case=False, na=False)),
"address_clean"] = '3333 WEST 20TH ST JACKSONVILLE, FL 32209'    

In [15]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("3333 20TH STREET", case=False, na=False)),
"address_clean"] = '3333 WEST 20TH ST JACKSONVILLE, FL 32209'

In [16]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("3333 20TH ST", case=False, na=False)),
"address_clean"] = '3333 WEST 20TH ST JACKSONVILLE, FL 32209'

In [17]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("^3625 U", case=False, na=False)),
"address_clean"] = '3625 UNIVERSITY BLVD S JACKSONVILLE, FL 32216'

In [18]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "3635 UNIVERSITY BLVD", case=False, na=False)),
"address_clean"] = '3625 UNIVERSITY BLVD S JACKSONVILLE, FL 32216'

In [19]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "3925 UNIVERSITY BLVD S", case=False, na=False)),
"address_clean"] = '3625 UNIVERSITY BLVD S JACKSONVILLE, FL 32216'

In [20]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "UNNIVERSITY", case=False, na=False)),
"address_clean"] = '3625 UNIVERSITY BLVD S JACKSONVILLE, FL 32216'

In [21]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains('BURY', case=False)) &
    (cases_2009_2015["PetitionerAddressPart1"].str.contains('^3947')),
"address_clean"] = '3947 SALISBURY RD. N.'

In [22]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains('3947 SALIS', case=False)),
"address_clean"] = '3947 SALISBURY RD. N.'

In [23]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("^630", case=False, na=False)) &
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("BEA", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [24]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "6300 BEACH", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [25]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "6300 BECH", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [26]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "6300 B3EACH BLVD", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [27]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "6300 BEEACH BLVD", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [28]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains(
            "3600 BEACH BLVD", case=False, na=False)),
"address_clean"] = '6300 BEACH BLVD JACKSONVILLE, FL 32216'

In [29]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("^655", case=False, na=False)),
"address_clean"] = '655 W 8TH STREET JACKSONVILLE, FL 32209'

In [30]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("655 W", case=False, na=False)),
"address_clean"] = '655 W 8TH STREET JACKSONVILLE, FL 32209'

In [31]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("665 W", case=False, na=False)),
"address_clean"] = '655 W 8TH STREET JACKSONVILLE, FL 32209'

In [32]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("UF AND SHANDS", case=False, na=False)),
"address_clean"] = '655 W 8TH STREET JACKSONVILLE, FL 32209'

In [33]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("^800", case=False, na=False)),
"address_clean"] = '800 PRUDENTIAL DRIVE JACKSONVILLE, FL 32207'

In [34]:
cases_2009_2015.loc[
    (cases_2009_2015["PetitionerAddressPart1"].str.contains("800 PR", case=False, na=False)),
"address_clean"] = '800 PRUDENTIAL DRIVE JACKSONVILLE, FL 32207'

## Petition count by institution

In [35]:
petition_counts = pd.DataFrame({
    "petition_count": cases_2009_2015.groupby(["address_clean","FileYear"]).size()
}).reset_index()

### River Point petitions over time (located at 6300 BEACH BLVD)

In [36]:
river_point = petition_counts[
        petition_counts["address_clean"] == "6300 BEACH BLVD JACKSONVILLE, FL 32216"
].set_index("FileYear")

In [37]:
river_point

,address_clean,petition_count
FileYear,,
2009,"6300 BEACH BLVD JACKSONVILLE, FL 32216",238
2010,"6300 BEACH BLVD JACKSONVILLE, FL 32216",549
2011,"6300 BEACH BLVD JACKSONVILLE, FL 32216",712
2012,"6300 BEACH BLVD JACKSONVILLE, FL 32216",1131
2013,"6300 BEACH BLVD JACKSONVILLE, FL 32216",1362
2014,"6300 BEACH BLVD JACKSONVILLE, FL 32216",940
2015,"6300 BEACH BLVD JACKSONVILLE, FL 32216",175


In [38]:
pct_change = (river_point.ix[2013]["petition_count"]-river_point.ix[2009]["petition_count"])\
    /float(river_point.ix[2009]["petition_count"]) * 100.0

In [39]:
print("The percentage change from 2009 to 2013 was: {0:.1f}%".format(pct_change))

The percentage change from 2009 to 2013 was: 472.3%


---

---

---